# EDA CropData

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import category_encoders as ce

In [ ]:
# Load a sheet into a DataFrame by its name
df = pd.read_excel('files/food-twentieth-century-crop-statistics-1900-2017-xlsx.xlsx', sheet_name="CropStats")

In [ ]:
# See basic info about our dataframe
df.info()

## Drop columns we don't need.

In [ ]:
df=df.drop(['Unnamed: 0','admin2','notes'], axis=1)
df.info()

In [ ]:
sizes = [df['admin1'].count(),len(df)-df['admin1'].count()]
plt.figure(figsize=(7,7))
plt.title('Region filled in')
plt.pie(sizes,labels=['filled in','not filled in'],autopct='%1.1f%%', textprops={'fontsize':13})
plt.show()

In [ ]:
df.loc[df['admin0'] == 'Belgium']

In [ ]:
df_BelgiumNullValues=df.loc[df['admin0'] == 'Belgium']
print('The amount of filled in regions in Belgium:',df_BelgiumNullValues['admin1'].count())

In [ ]:
df_replaceAdmin1 = df.copy()
df_replaceAdmin1['admin1'].fillna(df['admin0'], inplace=True)
df_replaceAdmin1

In [ ]:
df_transformed = df_replaceAdmin1.copy()

Hier gaan we kijken of er plekken zijn waar we de yield nog kunnen berekenen aan de hand van de hectaren en de production.

In [ ]:
df_transformed.loc[df_transformed['hectares (ha)'].notna()][df_transformed['production (tonnes)'].notna()][df_transformed['yield(tonnes/ha)'].isna()][df_transformed['hectares (ha)'] != 0]

We hebben de yield ingevuld waarbij de hectaren en de productie waardes hadden.

In [ ]:
for index, row in df_transformed.iterrows():
    if pd.notna(row['hectares (ha)']) and pd.notna(row['production (tonnes)']) and pd.isna(row['yield(tonnes/ha)']) and row['hectares (ha)'] != 0:
        df_transformed.at[index, 'yield(tonnes/ha)'] = row['production (tonnes)'] / row['hectares (ha)']
df_transformed

Hier kunnen we zien dat de yield 0 is. Dit komt omdat er ook geen production was ook al zijn er wel gewassen geplant (De gewassen kunnen bijvoorbeeld vernield zijn door een storm).

In [ ]:
df_transformed.loc[df_transformed['yield(tonnes/ha)'] == 0]

Waar de yield niet ingevuld was hebben we aan de hand van backwards fill deze kunnen opvullen. We hebben opgemerkt dat de meeste missing values zich voordoen in de vroegere jaren. Daarom hebben we voor backwards fill gekozen zodat als er van land veranderd wordt zodat er zo wijnig mogelijk met het andere land rekening gehouden wordt.

In [ ]:
df_transformed['yield(tonnes/ha)'].bfill(inplace=True)

df_transformed.head(20)

Nu hebben hectaren en production niet heel veel meerwaarde omdat de yield is wat we gaan willen predicten.

In [ ]:
df_transformed=df_transformed.drop(['hectares (ha)','production (tonnes)'], axis=1)
df_transformed